In [ ]:
%config IPCompleter.greedy=True

## Improved Pre-Processing: 
Stopwords are removed in the tweets in order to improve the performance of classification using Naive Bayes Model At line 25 using a predefined stopwords.Here we pre-process the 30k data. 

Stopwords-One of the major forms of pre-processing is to filter out useless data. In natural language processing, useless words (data), are referred to as stop words.

In [ ]:
import keras
import numpy as np
import pandas as pd
import os
import glob
import math
import re
df = tweets = pd.read_excel("./clean_healthcare30000.xlsx")
df = df[['TweetText','Polarity']]
stop_words = set(['a','about','above','after','again','against','all','am','an','and','any','are','aren\'t','as','at','be','because','been','before','being','below','between','both','but','by','can\'t','cannot','could','couldn\'t','did','didn\'t','do','does','doesn\'t','doing','don\'t','down','during','each','few','for','from','further','had','hadn\'t','has','hasn\'t','have','haven\'t','having','he','he\'d','he\'ll','he\'s','her','here','here\'s','hers','herself','him','himself','his','how','how\'s','i','i\'d','i\'ll','i\'m','i\'ve','if','in','into','is','isn\'t','it','it\'s','its','itself','let\'s','me','more','most','mustn\'t','my','myself','no','nor','not','of','off','on','once','only','or','other','ought','our','ours', 'ourselves','out','over','own','same','shan\'t','she','she\'d','she\'ll','she\'s','should','shouldn\'t','so','some','such','than','that','that\'s','the','their','theirs','them','themselves','then','there','there\'s','these','they','they\'d','they\'ll','they\'re','they\'ve','this','those','through','to','too','under','until','up','very','was','wasn\'t','we','we\'d','we\'ll','we\'re','we\'ve','were','weren\'t','what','what\'s','when','when\'s','where','where\'s','which','while','who','who\'s','whom','why','why\'s','with','won\'t','would','wouldn\'t','you','you\'d','you\'ll','you\'re','you\'ve','your','yours','yourself','yourselves'])

def processRow(row):
    tweet = row.lower()    #Lower case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)    #delete any url
    tweet = re.sub('@[^\s]+','',tweet) #delete any @Username
    tweet = re.sub('[\s]+', ' ', tweet)#Remove additional white spaces
    tweet = re.sub('[\n]+', ' ', tweet) #Remove not alphanumeric symbols white spaces
    tweet = re.sub(r'[^\w]', ' ', tweet) #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #Remove Digits
    tweet = re.sub(" \d+", '', tweet)
    tweet = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", tweet)
    tweet = tweet.replace(':)','')    #Remove :( or :)
    tweet = tweet.replace(':(','')
    tweet = tweet.strip('\'"')    #trim
    tweet = [word for word in tweet.split() if word not in stop_words and len(word) > 1]#Removes stopwords and single letter words
    return ''.join(str(e) + " " for e in tweet)
testing = np.array(list(df['TweetText'][:30000]))  
for x in range(0,testing.shape[0]):
    testing[x] = processRow(testing[x])
df.TweetText = testing

#df.to_csv("clean_stopwordsremoved_healthcaretweet30000.csv", index = False)

## Models: Naive Bayes (NB)/ Support Vector Machine (SVM)/ Logistic Regression (LR)
This is the code used to get he results excel sheet to compare the performance of the three models in order to choose the fastest and most accurate. The model chosen (NB) as it is the fastest and performed the best in these two criteria.
In order to use the code you should have the nltk and sklearn packages.<br>

The Evaluation Metrics used to test the models performance are skllearns: precision, recall, fscore(f1), and accuracy.
The Performance Metrics use to thes the models performance are in the terms of time taken to Build the Text Corpus, Vectorize the data and for the models to classify the overall record (15k/30k data).<br>

Naming Concention of the csv files:<br>
CSV files with stopwords:<br>
    Version1: 15k data set<br>
    clean_healthcaretweet750 - 750 labelled of 15k<br>
    clean_healthcaretweet1500 - 1500 labelled of 15k<br>
    clean_healthcaretweet2250 - 2250 labelled of 15k<br>
    clean_healthcaretweet3000 - 3000 labelled of 15k<br>
    Version2: 30k data set<br>
    clean_healthcaretweet750_30k - 750 labelled of 15k<br>
    clean_healthcaretweet1500_30k - 1500 labelled of 15k<br>
    clean_healthcaretweet2250_30k - 2250 labelled of 15k<br>
    clean_healthcaretweet3000_30k - 3000 labelled of 15k<br>

CSV files without stopwords:<br>
    Version1: 15k data set<br>
    clean_stopwordsremoved_healthcaretweet750_15k - 750 labelled of 15k<br>
    clean_stopwordsremoved_healthcaretweet1500_15k - 1500 labelled of 15k<br>
    Version2: 30k data set<br>
    clean_stopwordsremoved_healthcaretweet750_30k - 750 labelled of 30k<br>
    clean_stopwordsremoved_healthcaretweet750_30k - 1500 labelled of 30k<br>
Below is the code used to test different performance evaluations of the three models: test set=20%   

In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')
#Set Random seed
np.random.seed(500)


def calPerformanceofModels(path,label,max_features):
    # Add the Data using pandas
    start = time.time()
    Corpus = pd.read_csv(path,encoding='latin-1')
    Corpus['Polarity'] = Corpus['Polarity'].apply(str) #converts the float string into string/obj for processing
    Corpus.dropna()
    #print(Corpus.shape)
    # Step - 1a : Tokenization : In this each entry in the corpus will be broken into set of words
    Corpus['TweetText']= [word_tokenize(str(entry)) for entry in Corpus['TweetText']]
    # Step - 1b: Perfom Word Stemming/Lemmenting.
    # WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    for index,entry in enumerate(Corpus['TweetText']):
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
        # The final processed set of words for each iteration will be stored in 'text_final'
        Corpus.loc[index,'text_final'] = str(Final_words)
    #print(Corpus['text_final'].head())
    end = time.time()
    tok_time = end-start
    
    # Step - 2: Split the model into Train and Test Data set
    start  = time.time()
    Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['Polarity'],test_size=0.2)

    # Step - 3: Label encode the target variable  - This is done to transform Categorical data of string type in the data set into numerical values
    Encoder = LabelEncoder()
    Train_Y = Encoder.fit_transform(Train_Y)
    Test_Y = Encoder.fit_transform(Test_Y)
    # Step - 4: Vectorize the words by using TF-IDF Vectorizer - This is done to find how important a word in document is in comaprison to the corpus
    Tfidf_vect = TfidfVectorizer(max_features=max_features)
    Tfidf_vect.fit(Corpus['text_final'])

    Train_X_Tfidf = Tfidf_vect.transform(Train_X)
    Test_X_Tfidf = Tfidf_vect.transform(Test_X)
    # Step - 5: Now we can run different algorithms to classify our data check for accuracy
    end  = time.time()
    vect_time = end-start
    # Classifier - Algorithm - Naive Bayes
    # fit the training dataset on the classifier
    start  = time.time()
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Train_X_Tfidf,Train_Y)
    # predict the labels on validation dataset
    predictions_NB = Naive.predict(Test_X_Tfidf)
    end  = time.time()
    nb_time = end -start
    #NAIVE BAYES END
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    start  = time.time()
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf,Train_Y)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Test_X_Tfidf)
    end  = time.time()  
    svm_time = end -start
    #SVM END
    
    # Classifier - Algorithm - Logistic Regression
    # fit the training dataset on the classifier
    start  = time.time() 
    LogReg = LogisticRegression()
    LogReg.fit(Train_X_Tfidf,Train_Y)
    # predict the labels on validation dataset
    predictions_LR = LogReg.predict(Test_X_Tfidf)
    end = time.time()
    lr_time = end -start
    #lOGISTIC REGRESSION END

    #We compute the precison, recall and fscore of each of the models
    #We use Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for
    #label imbalance; it can result in an F-score that is not between precision and recall.

    prf_NB = precision_recall_fscore_support(predictions_NB, Test_Y,average='weighted')
    prf_SVM = precision_recall_fscore_support(predictions_SVM, Test_Y,average='weighted')
    prf_LR = precision_recall_fscore_support(predictions_LR, Test_Y,average='weighted')
    print("~~~~For N labels = ", label , ", Max features = ", max_features  ,"~~~~~\n")
    print("===Naive Bayes===\nPrecision, Recall, F1-Score: ",  prf_NB[0]*100,prf_NB[1]*100,prf_NB[2]*100 )
    print("Naive Bayes Accuracy Score -> ",accuracy_score(Test_Y, predictions_NB)*100)
    print("===SVM===\nPrecision, Recall, F1-Score: ",  prf_SVM[0]*100,prf_SVM[1]*100,prf_SVM[2]*100)
    print("SVM Accuracy Score -> ",accuracy_score(Test_Y, predictions_SVM)*100)
    print("===Logistic Regression===\nPrecision, Recall, F1-Score: ",  prf_LR[0]*100,prf_LR[1]*100,prf_LR[2]*100)
    print("LR Accuracy Score -> ",accuracy_score(Test_Y,predictions_LR )*100)
    print("Time spent on tokenizing for bag of words: ", tok_time, "\n")
    print("Time spent on vectorizing for NB/SVM/LR", vect_time, "\n")
    print("Time spent on predicting for NB/SVM/LR models (respectively): ", "\n",
          nb_time, "/", svm_time ,"/", lr_time,"\n")

## Tests Conducted: 
Variables being changed Max Features (number of important words in the text Corpus), Number of labelled Data and Total Number of Data (Scaling).<br>
The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.<br>

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.<br>

The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0. In our example we used a multiplier of 100 so as we can read it clearer.<br>

The F-beta score weights recall more than precision by a factor of beta. beta == 1.0 means recall and precision are equally important.<br>

The support is the number of occurrences of each class in predictions_xxx.<br>

If pos_label is None and in binary classification, this function returns the average precision, recall and F-measure if average is one of 'micro', 'macro', 'weighted' or 'samples'.<br>

## Test 1 (15k data set): There are Three variables that we vary to observe the performance of the three models: NB, SVM, LB 
For the first experiment we kept the Max Features and the total data the same.
Changing the labelled Data.<br>

Conclusion: Results is that as more labelled data is trained for classifying the 15k sets of tweets the evaluation metrics precision.recall/f1 and accuracy decreased significantly. However, a good effect is that the tokenizing time has decreased by roughly 5%. The time spent on vectorizing stayed roughly the same while classifying time for 15k data set has decreased as more labelled data is fed to it. 

Best to worst performing Model(evaluation and performance metrics): NB>>LB>>SVM

In [ ]:
calPerformanceofModels('./clean_healthcaretweet750.csv',750,5000)
calPerformanceofModels('./clean_healthcaretweet1500.csv',1500,5000)
calPerformanceofModels('./clean_healthcaretweet2250.csv',2250,5000)
calPerformanceofModels('./clean_healthcaretweet3000.csv',3000,5000)

## Test 3 (30k data set): There are Three variables that we vary to observe the performance of the three models: NB, SVM, LB 
For the third experiment we kept the Max Features and the total data the same.
Changing the labelled Data.<br>
Conclusion: Results is that as more labelled data is trained for classifying the 30k sets of tweets the evaluation metrics precision.recall/f1 and accuracy decreased roughly 4-7%. Moreover, the tokenizing time has increased roughly twice from 500s to 1000s. The time spent on vectorizing almost doubled as well. <br><br>
Performance metrics Classification time For Models (increase of labelled data from 750 to 3000 of 30k):<br>
NB: Decreased as labelled data increased<br>
SVM: Increased 10x as labbeled data increased <br>
LB: Stayed Roughly the Same<br>
Classification time For Models (increase of labelled data from 750 to 3000 of 30k):<br>
NB: Decreased as labelled data increased<br>
SVM: Increased 10x as labbeled data increased <br>
LB: Stayed Roughly the Same<br>

Best to worst performing Model : NB>>LB>>SVM<br>
In terms of time NB performed the fastest and the most accurate while LB/SVM performed around the same accuracy but worse than NB. SVM performed significantly slower than NB and LB and shouldn't used for scaling.

In [ ]:
calPerformanceofModels('./clean_healthcaretweet750_30k.csv',750,5000)
calPerformanceofModels('./clean_healthcaretweet1500_30k.csv',1500,5000)
calPerformanceofModels('./clean_healthcaretweet2250_30k.csv',2250,5000)
calPerformanceofModels('./clean_healthcaretweet3000_30k.csv',3000,5000)

## Test 2 (15k data set): Vary labelled data and Num of Max Features (MF), Keep total data the same.
Changed the Labelled data from 750,1500,2250 and 3000 respectively.<br>
Changed the Max Features from 2500,5000,7500 and 10000 respectively.<br>
The data will load qite slow roughly 400s to 1000s per call of calPerformanceofModels:<br>

Increasing MF increased the time it takes to tokenize, and vectorize the data for the same total number of data. From the test the MF value of roughly 5000 performed better thus 5000 was chosen for scaling.<br>

Time taken to tokenize and Vectorize data:<br>
NB/SVM/LR:Tokenizing and Vectorizing increased more than double<br>
Time taken to classify Data:<br>
NB: Increased x4 roughly<br>
SVM:  Increased x4 roughly<br>
LR: Increased x3 roughly<br>

Performarmance Metrics for models:
NB/SVM/LR: As labelled data increased from 750 to 3000 at the same MF it decreased significantly (roughly 10%-15% decrease), However increasing the MF together with the labelled data alleviate the decrease performance by the model.  <br>

Best to worst Performing Model (time taken/performance metrics): NB>>LB>>SVM<br>


In [ ]:
#750 labelled data, change MF 
calPerformanceofModels('./clean_healthcaretweet750.csv',750, 5000)
calPerformanceofModels('./clean_healthcaretweet750.csv',750, 2500)
calPerformanceofModels('./clean_healthcaretweet750.csv',750, 7500)
calPerformanceofModels('./clean_healthcaretweet750.csv',750, 10000)
#1500 labelled data, change MF 
calPerformanceofModels('./clean_healthcaretweet1500.csv',1500, 5000)
calPerformanceofModels('./clean_healthcaretweet1500.csv',1500, 2500)
calPerformanceofModels('./clean_healthcaretweet1500.csv',1500, 7500)
calPerformanceofModels('./clean_healthcaretweet1500.csv',1500, 10000)
#2250 labelled data, change MF 
calPerformanceofModels('./clean_healthcaretweet2250.csv',2250, 5000)
calPerformanceofModels('./clean_healthcaretweet2250.csv',2250, 2500)
calPerformanceofModels('./clean_healthcaretweet2250.csv',2250, 7500)
calPerformanceofModels('./clean_healthcaretweet2250.csv',2250, 10000)
#3000 labelled data, change MF 
calPerformanceofModels('./clean_healthcaretweet3000.csv',3000, 5000)
calPerformanceofModels('./clean_healthcaretweet3000.csv',3000, 2500)
calPerformanceofModels('./clean_healthcaretweet3000.csv',3000, 7500)
calPerformanceofModels('./clean_healthcaretweet3000.csv',3000, 10000)


## Test 4 (30k data set): Vary labelled data and Num of Max Features (MF), Keep total data the same. Testing on Larger Data.
Changed the Labelled data from 750,1500 and 3000 respectively.<br>
Changed the Max Features from 2500,5000,7500 and 10000 respectively.<br>

The difference between 15k and 30k data is that at 30k data the time taken to tokenize it stayed relatively the same as MF is increased which is good for scaling.

Time taken to tokenize and Vectorize data:<br>
NB/SVM/LR:Tokenizing and Vectorizing increased more than double<br>
Time taken to classify Data:<br>
NB: Increased x4 roughly<br>
SVM:  Increased x4 roughly<br>
LR: Increased x3 roughly<br>

Performarmance Metrics for models:
NB/SVM/LR: As labelled data increased from 750 to 3000 at the same MF it decreased significantly (roughly 10%-15% decrease), However increasing the MF together with the labelled data alleviate the decrease performance by the model.  <br>

Best to worst Performing Model (time taken/performance metrics): NB>>LB>>SVM<br><br>
Takeaway: at scaling number of labelled data matters, a lower number might be better as biasedness in data might be better for classification.

In [ ]:
#750 labelled data, change MF 
calPerformanceofModels('./clean_healthcaretweet750_30k.csv',750,2500)
calPerformanceofModels('./clean_healthcaretweet750_30k.csv',750,5000)
calPerformanceofModels('./clean_healthcaretweet750_30k.csv',750,7500)
calPerformanceofModels('./clean_healthcaretweet750_30k.csv',750,10000)
#1500 labelled data, change MF 
calPerformanceofModels('./clean_healthcaretweet1500_30k.csv',1500,2500)
calPerformanceofModels('./clean_healthcaretweet1500_30k.csv',1500,5000)
calPerformanceofModels('./clean_healthcaretweet1500_30k.csv',1500,7500)
calPerformanceofModels('./clean_healthcaretweet1500_30k.csv',1500,10000)
#2250 labelled data, change MF 
## Validate Results using clean_healthcaretweet1500 dataset
calPerformanceofModels('./clean_healthcaretweet3000_30k.csv',3000,2500)
calPerformanceofModels('./clean_healthcaretweet3000_30k.csv',3000,5000)
calPerformanceofModels('./clean_healthcaretweet3000_30k.csv',3000,7500)
calPerformanceofModels('./clean_healthcaretweet3000_30k.csv',3000,10000)
#3000 labelled data, change MF 


## Test 5 (30k data set): Effect of Removing Stopwords on Data, on the Three Models.
Changed the Labelled data from 750,1500 and 3000 respectively.<br>
Kept the Max Features at 5000.<br>

The difference between 15k and 30k data is that at 30k data the time taken to tokenize it stayed relatively the same as MF is increased which is good for scaling.

Time taken to tokenize and Vectorize data:<br>
NB/SVM/LR: After stopwords has been removed time take to tokenize and has signigicantly decreased to more than half (860s to 320s) while time taken to vectorize data increased as the number of labelled data and after removing stopwords from (0.98s to 1.03s) which is not very big. <br>
Time taken to classify Data:<br>
NB: Increased (from 0.0059s to 0.0069) as labelled data increased. Thus it is better to keep labelled data a bit smaller. <br>
SVM/LR:  For time to classify data both these models stayed almost the same with/without stopwords<br>

Performarmance Metrics for models:
NB/SVM/LR: As labelled data increased from 750 to 3000 at the same MF it decreased significantly (roughly 4-7% decrease), However increasing the MF together with the labelled data alleviate the decrease performance by the model.  <br>
Best to worst Performing Model (time taken/performance metrics): NB>>LB>>SVM<br><br>


Takeaway: Throughout all the test NB performed the best in terms of both time and performance and in terms of possible scaling. 

In [ ]:
#750
print("Before-----------------\n")
calPerformanceofModels('./clean_healthcaretweet750_30k.csv',750,5000)
print("After-----------------\n")
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_30k.csv',750,5000)
#1500
print("Before-----------------\n")
calPerformanceofModels('./clean_healthcaretweet1500_30k.csv',1500,5000)
print("After-----------------\n")
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet1500_30k.csv',1500,5000)
#3000
print("Before-----------------\n")
calPerformanceofModels('./clean_healthcaretweet3000_30k.csv',3000,5000)
print("After-----------------\n")
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet3000_30k.csv',3000,5000)


## Chosen Model: Naive Bayes scaling from 15k to 30k removing Stopwords
Kept the Labelled data at 750.<br>
Kept the Max Features 5000.<br>
Total data from 15k to 30k.<br>
Before/After Removing Stopwords.<br>


At before and after of 15k and 30k after removing the stopwords the NB model has scaled better while only losing minimal accuracy in the evaluation metrics: precision, recall,f1 and accuracy.<br>

Time taken to classify Data:<br>
NB: The time taken to classify the data after the removal of stopwords has scaled well from 15k to 30k data at (0.0098 to 0.050s) as compared to before removal (0.0051 to 0.0099s). Which means that removing stopwords will help in scaling data as it help make the model scale the classification time of the tweets faster.

Performarmance Metrics for models:
NB: As the number of total data increased from 15k to 30k while maintaining a small labelled data for training the precision, recall and accuracy has increased. 

Takeaway: This Naive Bayes with improved pre-processing has managed to improve the performance metrics of the model through the stopwords removal and scales well as the number of data to be process increased.

In [ ]:
print("15k with stopwords-----------------\n")
calPerformanceofModels('./clean_healthcaretweet750.csv',750,5000)
print("15k without stopwords-----------------\n")
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_15k.csv',750,5000)
print("30k with stopwords-----------------\n")
calPerformanceofModels('./clean_healthcaretweet750_30k.csv',750,5000)
print("30k without stopwords-----------------\n")
print("30k-----------------\n")
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_30k.csv',750,5000)

## Chosen Model: Naive Bayes scaling from 15k to 30k increasing MF
Kept the Labelled data at 750.<br>
Increasing the Max Features from 5000 to 7500 to 10000.<br>
Total data from 15k to 30k.<br>
After Removing Stopwords.<br>

At 15k data set we have found that optimum MF = 7500 and at 30k MF = 10000. 

In [ ]:
print("15k without stopwords MF increased-----------------\n")
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_15k.csv',750,5000)
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_15k.csv',750,7500)
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_15k.csv',750,10000)
print("30k without stopwords MF increased-----------------\n")
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_30k.csv',750,5000)
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_30k.csv',750,7500)
calPerformanceofModels('./clean_stopwordsremoved_healthcaretweet750_30k.csv',750,10000)

## Conclusion:
Naive Bayes performed the best and when scaling MF should be optimized (it might be increased or decreased) to find the best performance in terms of time and evaluation metrics.  At this example 